In [ ]:
from ast import literal_eval
from copy import deepcopy
import datetime as dt
from functools import partial, reduce
from inspect import signature
import os
from operator import or_, and_, contains
import random as rand
import subprocess

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, MATCH, ALL
from dash.exceptions import PreventUpdate
from dateutil.parser import parse
import django
from django.db.models import Q
import flask
from flask_caching import Cache
#from jupyter_plotly_dash import JupyterDash
import pandas as pd
import PIL
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go
import pylibmc
from toolz import keyfilter, valfilter, merge, isiterable, get_in

# to make jupyter-plotly-dash work, had to install jupyter_server_proxy explicitly
# pip install jupyter_server_proxy
# jupyter serverextension enable jupyter_server_proxy
# but honestly it sort of isn't very good anyway and is not presently being used

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mastspec.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from plotter.components import *
from plotter.models import *
from plotter.graph import *
# from plotter.views import *
# from plotter.forms import *
from utils import (djget, modeldict, rows, columns, qlist, keygrab, in_me,
                   pickitems, pickctx, make_printer, dump_it, get_parameters,
                  partially_evaluate_from_parameters, ctxdict, not_triggered, 
                   trigger_index, listify, triggered_by)

In [ ]:
# start memcached server and initialize a client for caching
# make sure memcached is running with correct parameters
# ,
# systemctl edit memcached.service --full , etc
# this can easily be set to start at runtime in a container

client = pylibmc.Client(["127.0.0.1"], binary=True)

In [ ]:
app = dash.Dash()

In [ ]:
# we're using this to create thread-safe global values.
# specifically, dash does not work well with setting Python globals within 
# the execution tree of a callback.
# also caching relatively expensive things like database lookups to the filesystem.
# is this actually slower? maybe. if so we will supplement with storing
# data in divs.
# we could also instead consider memoizing using lru_cache or similar
# note that we are _not_ presently attempting to make this a multiuser application,
# but a separate cache per user might easily allow that.

# this is separate from in-browser persistence we will likely add later to 
# protect against, e.g., stray page refreshes
CACHE_CONFIG = {
    'CACHE_TYPE': 'memcached', 
    'CACHE_DIR': './.cache',
    'CACHE_DEFAULT_TIMEOUT': 0,
    'SERVERS':client
}
cache = Cache()
cache.init_app(app.server, config=CACHE_CONFIG)
cache.clear()
cset = cache_set(cache)
cget = cache_get(cache)

In [ ]:
fig = main_graph()
spec_model = MSpec
# active queryset is explicitly stored in global cache, as are
# other app runtime values
cset('queryset', spec_model.objects.all().prefetch_related("observation"))
cset('open_graph_viewers', [])
# these lists of input objects are defined here primarily to avoid excessive
# repetition in app structure definition and function calls.
x_inputs = [
    Input('filter-1-x', 'value'),
    Input('filter-2-x', 'value'),
    Input('filter-3-x', 'value'),
    Input('axis-option-x', 'value'),
]
y_inputs = [
    Input('filter-1-y', 'value'),
    Input('filter-2-y', 'value'),
    Input('filter-3-y', 'value'),
    Input('axis-option-y', 'value')
]

static_image_url = '/images/'
image_directory =  './static_in_pro/our_static/img/'

In [ ]:
# insert 'global' values for this app into callback functions.
# note that insertion of spec_model into functions may end up being
# a way to generate separate function 'namespaces'
# in the possible case of wanting to mix mastcam / z data 
# within a single app instance.
# typically our convention is that 'global' variables in these functions
# are keyword-only arguments and callback inputs / states are positional
# arguments.
update_spectrum_images = cache.memoize()(update_spectrum_images)
update_spectrum_graph = cache.memoize()(update_spectrum_graph)

settings = {
        'x_inputs': x_inputs,
        'y_inputs': y_inputs,
        'cget': cget,
        'cset': cset,
        'graph_function': main_graph_scatter,
        'spec_graph_function': mspec_graph_line,
        'spec_model': MSpec,
        'image_directory':  image_directory,
        # scale factor, in viewport units, for spectrum images
        'base_size': 20,
        'static_image_url': static_image_url,
        # file containing saved searches
        'search_file': './saves/saved_searches.csv'
    }
functions_requiring_settings = [
    control_tabs,
    control_search_dropdowns,
    recalculate_graph, 
    update_search_options,
    update_queryset,
    change_calc_input_visibility,
    toggle_search_input_visibility,
    update_spectrum_graph,
    graph_point_to_metadata,
    update_spectrum_images,
    populate_saved_search_drop,
    save_search_tab_state
]
for function in functions_requiring_settings:
    globals()[function.__name__] = partially_evaluate_from_parameters(
        function, settings
    )

In [ ]:
# memoize functions that seem to want caching
# this isn't totally working because the local version isn't mostly what's
# actually being called. doesn't deal with checks in handle_search, etc
# not a big deal right now, will make it work later if necessary
# for function in [
#         recalculate_graph,
#         handle_graph_search,
#         qlist,
#         make_axis,
#         update_search_options,
#         update_queryset
#     ]:
#     function = cache.memoize()(function)

In [ ]:
# https://community.plotly.com/t/serve-locally-option-with-additional-scripts-and-style-sheets/6974/6

# serve static images using a flask route.
# does defining this function here violates my conventions a little bit? not sure. 
@app.server.route(static_image_url+'<path:path>')
def static_image_link(path):
    static_folder = os.path.join(os.getcwd(), image_directory)
    return flask.send_from_directory(static_folder, path)

In [ ]:
# app layout definition 

app.layout = html.Div(children = [
    dcc.Tabs(
        children = [search_tab(spec_model)],
        value = 'main_search_tab',
        id = 'tabs'
    ),
])

In [ ]:
# callback creation section: register functions with app i/o

for axis in ['x', 'y']:
    app.callback(
        [
            Output('filter-1-'+axis, 'style'),
            Output('filter-2-'+axis, 'style'),
            Output('filter-3-'+axis, 'style'),
        ],
        [Input('axis-option-'+axis, 'value')]
    )(change_calc_input_visibility)

app.callback(
    Output('main-graph', 'figure'),
    # maybe later add an explicit recalc button?
    [
        *x_inputs, 
        *y_inputs, 
        Input({'type':'search-trigger', 'index':ALL}, 'value'),
        Input('main-graph','hoverData')
    ],
    [
        State('main-graph','figure')
    ]
)(recalculate_graph)

app.callback(
    [
        Output({'type':'term-search', 'index':MATCH},'style'),
        Output({'type':'number-search', 'index':MATCH},'style'),
    ],
    [Input({'type':'field-search', 'index':MATCH},'value')],
    )(toggle_search_input_visibility)

app.callback(
    [
        Output({'type':'term-search', 'index':MATCH},'options'),
        Output({'type':'number-range-display','index':MATCH},'children'),
        Output({'type':'number-search', 'index':MATCH},'value'),
    ],
    [
        Input({'type':'field-search', 'index':MATCH},'value'),
        Input({'type':'load-trigger', 'index':0}, 'value')
    ],
    [
        State({'type':'number-search', 'index':MATCH},'value'),
    ]
    )(update_search_options)

app.callback(
        Output({'type':'search-trigger', 'index':0},'value'),
        [
            Input({'type':'submit-search', 'index':ALL},'n_clicks'),
            Input({'type':'load-trigger', 'index':0}, 'value')
        ],
        [
            State({'type': 'field-search', 'index': ALL}, 'value'),
            State({'type': 'term-search', 'index': ALL}, 'value'),
            State({'type': 'number-search', 'index': ALL}, 'value'),
            State({'type':'search-trigger', 'index':0},'value')
        ])(update_queryset)

app.callback(
    [
        Output('search-container', 'children'),
        Output({'type':'submit-search', 'index':1},'n_clicks')
    ],
    [
        Input('add-param', 'n_clicks'), 
        Input({'type':'remove-param', "index":ALL},'n_clicks')
    ],
    [
        State('search-container', 'children'),
        State({'type':'submit-search', 'index':1}, 'n_clicks')
    ]
)(control_search_dropdowns)

# make graph viewer tabs

app.callback(
    [
        Output('tabs', 'children'), 
        Output('tabs', 'value'),
        Output({'type':'load-trigger', 'index':0}, 'value')
    ],
    [
        Input('viewer-open-button','n_clicks'),
        Input({'type':'tab-close-button', "index":ALL},'n_clicks'),
        Input('load-search-load-button', 'n_clicks')
    ],
    [
        State('tabs','children'),
        State('load-search-drop', 'value'),
        State({'type':'load-trigger', 'index':0}, 'value')
    ]
)(control_tabs)

#debug printer
# app.callback(
#     Output('fake-output-for-callback-with-only-side-effects-1', 'children'),
#     [Input('load-search-drop', 'value')]
# )(print_callback)


# right now main and view graph hover functions are basically duplicates, 
# but i'm reserving the possibility that they'll have different behaviors later

app.callback(
    Output({'type':'main-spec-image', 'index':0}, "children"), 
    [Input('main-graph', "hoverData")]
    )(update_spectrum_images)

app.callback(
    Output({'type':'main-spec-print', 'index':0}, "children"), 
    [Input('main-graph', "hoverData")]
    )(graph_point_to_metadata)

app.callback(
    Output({'type':'main-spec-graph','index':0},'figure'),
    [Input('main-graph','hoverData')]
)(update_spectrum_graph)

app.callback(
    Output({'type':'view-spec-image', 'index':MATCH}, "children"), 
    [Input({'type':'view-graph', "index":MATCH},'hoverData')]
    )(update_spectrum_images)

app.callback(
    Output({'type':'view-spec-print', 'index':MATCH}, "children"), 
    [Input({'type':'view-graph', "index":MATCH},'hoverData')]
    )(graph_point_to_metadata)

app.callback(
    Output({'type':'view-spec-graph','index':MATCH},'figure'),
    [Input({'type':'view-graph', "index":MATCH},'hoverData')]
)(update_spectrum_graph)

app.callback(
    Output('fake-output-for-callback-with-only-side-effects-0', 'children'),
    [Input('load-search-save-button', 'n_clicks')]
)(save_search_tab_state)


app.callback(
    Output('load-search-drop', 'options'),
    [Input('load-search-save-button', 'n_clicks')]
    )(populate_saved_search_drop)

# def dummyfunc(data,figure):
#     hovered_point = data["points"][0]["customdata"]
#     figure.update_traces(
#        marker={'color':'red'}
#     )
#     return figure

In [ ]:
app.run_server(debug=True, use_reloader = False)

In [ ]:
MObs.objects.filter(sol__exact=2029.0)

In [ ]:
app.callback_map

In [ ]:
def robj(model):
    return rand.choice(model.objects.all())

In [ ]:
parameters = [
    {'field': '', 'term': 's', 'begin': '', 'end': ''}, 
    {'field': 's', 'term': '', 'begin': '', 'end': ''}
]


In [ ]:
MObs.objects.all().values_list("sol")

In [ ]:
MObs.objects.all().filter(reduce(or_,[Q(**{"sol__iexact":2038}), Q(**{"sol__iexact":2043})]))